# CAI Jokes

In [1]:
import re
from get_llm_response import get_llm_response, Model


In [19]:
model = Model.claude_v1_2

num_samples = 5

constitution_rules = [
    "Instructions must explicitly describe the tone that the editor adpots.",
    "Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.",
    "Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:\n\n<article>ARTICLE_CONTENT</article>\n\n[Rest of instruction goes here]'.",
    "Instructions which involve multiple steps should be broken down into numbered subtasks.",
    # "Instructions must include explicit examples. For example, if the instruction is to check for spelling errors, it should include an example typo and its correction.",
]
constitution_str = "\n".join(map(lambda x: f"{x[0]+1}. {x[1]}", enumerate(constitution_rules)))


def get_concepts_prompt(n: int):
    return f"Generate a list of {n} tasks that a Copy Editor might do as part of their job, given a written article. The output of each completed task should be written text. Write each task on a separate, numbered line."


def get_naive_response_prompt(concept: str):
    return f"""I have an AI agent which acts as a Copy Editor and I want it to complete the following task:

<task>
{concept}
</task>

The agent only responds with written text. Write a concise instruction for the agent, asking it to complete this task. Don't include any preamble, just respond directly with the instruction for the agent."""


def get_critique_prompt(naive_response: str):
    return f"""Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
{naive_response}
</instruction>

There may be some problems with this instruction. In particular, the instruction must abide by the following rules:

<rules>
{constitution_str}
</rules>

List each rule that the instruction breaks. State the rule verbatim, then describe how the instruction breaks the rule.

For example, if the instruction breaks rule 1, you would write:

The instruction breaks the following rules:
1. Rule: {constitution_rules[0]} - Reason: ..."""


def get_rewrite_prompt(naive_response: str, critique: str):
    return f"""Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
{naive_response}
</instruction>

The instruction is supposed to follow certain rules, but it breaks them as follows:

<issues>
{critique}
</issues>

Rewrite the instruction to address these issues. Respond directly with the rewritten instruction, without any preamble and without <introduction> tags."""


## Generate concepts

In [20]:
concepts_string = get_llm_response(get_concepts_prompt(num_samples), model, 0.7)
print(concepts_string)


 
1. Proofread the article for any spelling, grammar, or punctuation errors. 
2. Ensure consistent style and formatting, like heading styles, throughout the article. 
3. Flag any unclear or ambiguous language, wording, or phrases in the article. 
4. Suggest alternative wording or reorganization to improve the flow or clarity of ideas. 
5. Query the author for any facts or claims that need verification or citation of sources.


In [21]:
def extract_items(text):
    items = []
    lines = text.split('\n')
    for line in lines:
        stripped = line.strip()
        if stripped and re.match(r'^\d+\.\s', stripped):
            items.append(re.sub(r'^\d+\.\s', '', stripped))
    return items


concepts = extract_items(concepts_string)
if len(concepts) != num_samples:
    raise Exception(f"Expected {num_samples} concepts, but got {len(concepts)}")
else:
    print(f"Got {len(concepts)} concepts")


Got 5 concepts


## Generate naive responses

> **Terminology** - here we're prompting an LLM to generate prompts, which is confusing. We'll use the word "prompt" to refer to the initial input, and "response" to refer to the output

In [22]:
def get_naive_response(concept):
    return get_llm_response(get_naive_response_prompt(concept), model, 0.3)


# Concurrently (speeds up openai responses, not possible with anthropic)
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     naive_responses = list(executor.map(lambda x: get_naive_response(
#         x[0], x[1]), [(concept, i) for i, concept in enumerate(concepts)]))

# Sequentially
naive_responses = []
for concept in concepts:
    naive_responses.append(get_naive_response(concept))

tasksResponsesString = "\n\n\n===\n\n\n".join(
    [f"TASK:\n{task}\n\nNAIVE RESPONSE:\n{response}" for task, response in zip(concepts, naive_responses)])
print(tasksResponsesString)


TASK:
Proofread the article for any spelling, grammar, or punctuation errors.

NAIVE RESPONSE:
 Proofread the article for any spelling, grammar, or punctuation errors.


===


TASK:
Ensure consistent style and formatting, like heading styles, throughout the article.

NAIVE RESPONSE:
 Ensure consistent heading styles, font choices, spacing, capitalization, punctuation, and other formatting elements throughout the article.


===


TASK:
Flag any unclear or ambiguous language, wording, or phrases in the article.

NAIVE RESPONSE:
 Flag any unclear or ambiguous language, wording, or phrases in the article.


===


TASK:
Suggest alternative wording or reorganization to improve the flow or clarity of ideas.

NAIVE RESPONSE:
 Suggest improvements to the flow and clarity of this paragraph: 

The advent of artificial intelligence has brought significant opportunities and challenges. Machines can now perform human tasks like driving cars, translating languages, and diagnosing diseases. This is an

Most of these responses are slightly more detailed rewrites of the corresponding task. This is a good start, but ultimately not very useful and doesn't demonstrate most of the good practices in prompt design.

## Critique

Here's what an example prompt looks like:

In [24]:
print(get_critique_prompt(naive_responses[0]))


Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
 Proofread the article for any spelling, grammar, or punctuation errors.
</instruction>

There may be some problems with this instruction. In particular, the instruction must abide by the following rules:

<rules>
1. Instructions must explicitly describe the tone that the editor adpots.
2. Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.
3. Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:

<article>ARTICLE_CONTENT</article>

[Rest of instruction goes here]'.
4. Instructions which involve multiple steps should be broken down into numbered subtasks.
</rules>

List each rule that the instruction breaks. State the rule verbatim, then describe how the instruction breaks the rule.

For example, if the instruction breaks rule 1, you would write

In [23]:
def get_critique(naive_response):
    return get_llm_response(get_critique_prompt(naive_response), model, 0)

# Concurrently:
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     critiques = list(executor.map(lambda x: get_critique(x[0], x[1]), [
#                      (response, i) for i, response in enumerate(naive_responses)]))


# Sequentially:
critiques = []
for naive_response in naive_responses:
    critiques.append(get_critique(naive_response))

responseCritiqueString = "\n\n\n===\n\n\n".join(
    [f"NAIVE RESPONSE:\n{response}\n\nCRITIQUE:\n{critique}" for response, critique in zip(naive_responses, critiques)])
print(responseCritiqueString)


Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
 Proofread the article for any spelling, grammar, or punctuation errors.
</instruction>

There may be some problems with this instruction. In particular, the instruction must abide by the following rules:

<rules>
1. Instructions must explicitly describe the tone that the editor adpots.
2. Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.
3. Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:

<article>ARTICLE_CONTENT</article>

[Rest of instruction goes here]'.
4. Instructions which involve multiple steps should be broken down into numbered subtasks.
</rules>

List each rule that the instruction breaks. State the rule verbatim, then describe how the instruction breaks the rule.

For example, if the instruction breaks rule 1, you would write

## Rewrite

In [14]:
def get_rewrite(naive_response, critique):
    return get_llm_response(get_rewrite_prompt(naive_response, critique), model, 0.5)


# Concurrently:
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     rewrites = list(executor.map(lambda x: get_rewrite(x[0], x[1], x[2]), [
#         (response, critique, i) for i, (response, critique) in enumerate(zip(naive_responses, critiques))]))

# Sequentially:
rewrites = []
for i, (naive_response, critique) in enumerate(zip(naive_responses, critiques)):
    rewrites.append(get_rewrite(naive_response, critique))

rewriteString = "\n\n\n===\n\n\n".join(
    [f"NAIVE RESPONSE:\n{response}\n\nREWRITE:\n{rewrite}" for response, rewrite in zip(naive_responses, rewrites)])
print(rewriteString)


NAIVE RESPONSE:
 Proofread the text for spelling and grammar errors and provide corrections and suggestions.

REWRITE:
 Proofread the following article:  

<article>ARTICLE_CONTENT</article>

1. Check for spelling errors and provide up to 3 suggested corrections for each in up to 50 words total. Adopt a helpful and constructive tone.

2. Check for grammatical errors and provide up to 3 suggested corrections for each in up to 50 words total. Adopt a helpful and constructive tone.

Proofread the text for spelling and grammar errors and provide corrections and suggestions.


===


NAIVE RESPONSE:
 Verify that all numeric information and factual claims are correct.

REWRITE:
 <instruction>
Here is an article:
<article>ARTICLE_CONTENT</article>  

1. Verify that all numeric information and factual claims within the first 100 words are correct in a polite and helpful tone, aiming for a 50 word response.  

2. Verify that all numeric information and factual claims within the next 100 words ar